# WTI and DXY correlation 
- verify the assumption that since WTI - like Brent - is denominated in US dollars globally, 
a stronger dollar pressures crude prices.

Roadmap :
- Get WTI data : https://www.eia.gov/dnav/pet/pet_pri_spt_s1_d.htm
- Get DX-Y data -> USDX instead
- Join on date with maximum history
- Compute Pearson correlation
- Assess weather the correlation coefficient is relevant 
- Project out USDX onto WTI to get a first model of the crude idiosyncratic price

In [ ]:
import json
import requests
from pathlib import Path
import numpy as np
import pandas as pd
import polars as pl
import pandas_datareader as pdr
from matplotlib import pyplot as plt
from scipy.stats import pearsonr, probplot
from statsmodels.tsa.api import adfuller, kpss, VAR
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

raw_data_folder = Path("../raw_data")
commo_folder = raw_data_folder / "commo"
commo_folder = commo_folder.resolve()
print(f"Raw data location : {commo_folder}\n")
print("Current data files :")
for p in list(commo_folder.glob("*")):
    print(f"\t- {p}")

Raw data location : /home/leonc/code/fixed-income-analytics/raw_data/commo

Current data files :
	- /home/leonc/code/fixed-income-analytics/raw_data/commo/Cushing_OK_WTI_Spot_Price_FOB.csv


## Load data

### WTI

In [ ]:
# contains header

with open(commo_folder / "Cushing_OK_WTI_Spot_Price_FOB.csv") as f:
    for i in range(10):
        print(f.readline())

Cushing OK WTI Spot Price FOB

https://www.eia.gov/dnav/pet/hist/RWTCD.htm

09:19:15 GMT+0100 (British Summer Time)

Data Source: Thomson Reuters

Day,Cushing OK WTI Spot Price FOB  Dollars per Barrel

07/14/2025,68.19

07/11/2025,69.63

07/10/2025,67.78

07/9/2025,69.61

07/8/2025,69.55



In [ ]:
wti = (
    pl.scan_csv(commo_folder / "Cushing_OK_WTI_Spot_Price_FOB.csv", skip_rows=4)
    .with_columns(pl.col("Day").str.to_date("%m/%d/%Y"))
    .rename(
        {"Cushing OK WTI Spot Price FOB  Dollars per Barrel": "WTI spot", "Day": "date"}
    )
)

In [ ]:
_ = (
    wti.collect()
    .to_pandas()
    .set_index("date")
    .plot(
        figsize=(10, 4),
        grid=True,
        ylabel="(USD/b)",
        title="WTI historical daily spot price",
    )
)

### DX-Y / USDX / Dixie

In [ ]:
# Yahoo finance decomissioned
# history not available in alphavantage

dxy_ticker = "DX-Y.NYB"
alternative_ticker = "USDX"

ALPHAVANTAGE_API_KEY = "FTGVVCR5TZVEY9ZH"

In [ ]:
for ticker in [dxy_ticker, alternative_ticker]:
    search_url = f"https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords={ticker}&apikey={ALPHAVANTAGE_API_KEY}"
    search_result = requests.get(search_url).json()
    print(json.dumps(search_result, indent=2) + "\n")


quote_url = f"https://www.alphavantage.co/query?function=HISTORICAL_OPTIONS&symbol={alternative_ticker}&apikey={ALPHAVANTAGE_API_KEY}&date=2023-06-25"
r = requests.get(quote_url)
data = r.json()

print(json.dumps(data, indent=2))

{
  "bestMatches": []
}

{
  "bestMatches": [
    {
      "1. symbol": "USDX",
      "2. name": "SGI ENHANCED CORE ETF ",
      "3. type": "ETF",
      "4. region": "United States",
      "5. marketOpen": "09:30",
      "6. marketClose": "16:00",
      "7. timezone": "UTC-04",
      "8. currency": "USD",
      "9. matchScore": "1.0000"
    }
  ]
}

{
  "endpoint": "Historical Options",
  "message": "No data for symbol USDX on date 2023-06-25. Please specify a valid combination of symbol and trading day.",
  "data": []
}


### Fetching data from local proxy API service

In [ ]:
!podman ps

CONTAINER ID  IMAGE                                                             COMMAND               CREATED        STATUS        PORTS                   NAMES
6bc63067b4a0  6199185dbc2a16d93e0271263a1ba657a0481e1ad159c872c3d58b5f24f6e061  uvicorn --host 0....  5 minutes ago  Up 5 minutes  0.0.0.0:8000->8000/tcp  jolly_bartik


In [ ]:
!podman inspect 6bc63067b4a0 | grep Network
!podman inspect 6bc63067b4a0 | grep -i ip

print("\nIP is localhost since container is not WSL but running in main OS Ubuntu")

          "NetworkSettings": {
               "NetworkMode": "slirp4netns",
               "IPAddress": "",
               "IPPrefixLen": 0,
               "IPv6Gateway": "",
               "GlobalIPv6Address": "",
               "GlobalIPv6PrefixLen": 0,
               "LinkLocalIPv6Address": "",
               "LinkLocalIPv6PrefixLen": 0,
                              "HostIp": "",
                    "org.opencontainers.image.description": "An extremely fast Python package and project manager, written in Rust.",
                              "HostIp": "",
               "IpcMode": "shareable",

IP is localhost since container is not WSL but running in main OS Ubuntu


In [ ]:
# check that the API service is available (running in podman localhost)
STATE = "dev"

ipx = {"API": {"dev": "http://127.0.0.1:8000/", "staging": "http://0.0.0.0:8000/"}}

# query max dollar index history with pandas_datareader API (from FRED)

usdx_url = ipx["API"][STATE] + "pdr/usdx"

res = requests.get(usdx_url)
if res.status_code == 404:
    print("Service not available")

else:
    usdx = pd.DataFrame(json.loads(res.json()))
    usdx["date"] = usdx["date"].apply(lambda d: pd.Timestamp(d, unit="ms")).dt.date
    usdx = usdx.set_index("date")

## Verify the correlation assumption

In [ ]:
df = wti.join(
    pl.from_pandas(usdx.reset_index()).with_columns(pl.col("date").dt.date()).lazy(),
    on="date",
    how="inner",
)

print(f"Number of days joined : {df.select(pl.len()).collect().item()}")

Number of days joined : 4902


In [ ]:
df_p = df.collect().to_pandas().set_index("date")

_, ax = plt.subplots(figsize=(10, 4))
ax_twin = ax.twinx()
df_p["WTI spot"].plot(ax=ax, ylabel="USD")
df_p["USDX"].plot(ax=ax_twin, c="orange", grid=True, legend=True)
_ = ax.set_title("USDX vs WTI spot")

In [ ]:
print(
    f"Pearson correlation coefficient : {
df.select(
    pl.corr(
        pl.col("WTI spot"), 
        pl.col("USDX"),
        method="pearson"
    )
).collect().item()
}"
)

Pearson correlation coefficient : -0.48840042991376165


In [ ]:
df_no_nans = df.drop_nulls()
pr = pearsonr(
    df_no_nans.select(pl.col("WTI spot")).collect().to_numpy().T[0],
    df_no_nans.select(pl.col("USDX")).collect().to_numpy().T[0],
)
print(f"Pearson coefficient : {np.round(pr.statistic, decimals=3)}")
print(f"p-value : {np.round(pr.pvalue, decimals=5)}")
rho = pr.statistic

Pearson coefficient : -0.488
p-value : 0.0


## Quantify explainable variance

In [ ]:
wti_np = df_no_nans.select(pl.col("WTI spot")).collect().to_numpy().T[0]
usdx_np = df_no_nans.select(pl.col("USDX")).collect().to_numpy().T[0]

wti_unit = (wti_np - np.mean(wti_np)) / np.std(wti_np)
usdx_unit = (usdx_np - np.mean(usdx_np)) / np.std(usdx_np)

std = df.select(pl.exclude("date").std()).collect()
mean = df.select(pl.exclude("date").mean()).collect()

### In-sample linear USDX projection

In [ ]:
(osm_w, osr_w), (a_w, b_w, _) = probplot(wti_np, dist="norm", fit=True)
(osm_u, osr_u), (a_u, b_u, _) = probplot(usdx_np, dist="norm", fit=True)

In [ ]:
fig, axs = plt.subplots(figsize=(10, 4), ncols=2)
axs[0].scatter(osm_w, osr_w, s=1.0)
axs[1].scatter(osm_u, osr_u, s=1.0)
m_w, M_w = np.min(osm_w), np.max(osm_w)
m_u, M_u = np.min(osm_u), np.max(osm_u)
x_w = np.linspace(m_w, M_w, 100)
x_u = np.linspace(m_u, M_u, 100)
axs[0].plot(x_w, b_w + a_w * x_w, c="r")
axs[1].plot(x_u, b_u + a_u * x_u, c="r")
axs[0].grid()
axs[1].grid()
axs[0].set_title("WTI")
axs[1].set_title("USDX")
_ = fig.suptitle("QQ-Plot")

While WTI prices taken as independant samples are approximately Gaussian, this assumption is not verified for the dollar index.

> The p-value of the Pearson coefficient is not reliable since the pair of variables is not a bivariate Gaussian.

In [ ]:
res = wti_unit - rho * usdx_unit

r2 = 1.0 - np.var(res)  # wti with unit variance

print("R2 coefficient :", np.round(r2, decimals=2))

R2 coefficient : 0.2


The in-sample explainable variance is 24 %.

However the linear projection approximation is not satisfying, 
first because the time series are not stationary 
so the correlation coefficient is mis-specified,
then because it is not robust to noise and scale 
(the result depends on the selected time-frame).

A better suited approach must involve the time-structure of the data.
In the next section, time-series analysis models are fitted to increase the amount of explainable variance of WTI from USDX.

### Looking at the increments

The series are not stationary since they don't have a constant mean.
One can test whether it is difference-stationary.



In [ ]:
EPS = 0.1  # clip negative prices

df_increments = df.with_columns(
    pl.exclude("date").clip(lower_bound=EPS).log().diff()
).drop_nulls()

df_increments = df_increments.with_columns(
    pl.col("WTI spot").clip(
        upper_bound=pl.quantile("WTI spot", 0.99),
        lower_bound=pl.quantile("WTI spot", 0.01),
    ),
    pl.col("USDX").clip(
        upper_bound=pl.quantile("USDX", 0.99), lower_bound=pl.quantile("USDX", 0.01)
    ),
)

df_increments = df_increments.with_columns(
    (pl.col("WTI spot") - pl.mean("WTI spot")) / pl.std("WTI spot"),
    (pl.col("USDX") - pl.mean("USDX")) / pl.std("USDX"),
)


print(f"Number of days joined : {df_increments.select(pl.len()).collect().item()}")


df_p = df_increments.collect().to_pandas().set_index("date")

wti_incr_unit = df_p["WTI spot"]
usdx_incr_unit = df_p["USDX"]

Number of days joined : 4789


In [ ]:
_, ax = plt.subplots(figsize=(10, 4))

datex = df_p.index
ax.plot(datex, wti_incr_unit, alpha=0.8)
ax.plot(datex, usdx_incr_unit, c="orange", alpha=0.5)
ax.grid()
_ = ax.set_title("USDX vs WTI spot log diffs")

In [ ]:
print(
    f"Pearson correlation coefficient : {
df_increments.select(
    pl.corr(
        pl.col("WTI spot"), 
        pl.col("USDX"),
        method="pearson"
    )
).collect().item()
}"
)

Pearson correlation coefficient : -0.29585351751962435


The coefficient is lower than the non-differenciated time-series.

It is a better estimation of how well the USDX can account for variation in WTI value, 
since it measures **by how much both time series move in the same direction simultneously**
rather than measuring **by how much both series are above their global mean at the same time** 
which is more descriptive than predictive.

### Lead-lag ?

In [ ]:
for i in range(5)[::-1]:
    print("\nDays of lag : ", i)
    pr = pearsonr(wti_incr_unit[i:], usdx_incr_unit[:-i] if i > 0 else usdx_incr_unit)
    print(f"Pearson coefficient : {np.round(pr.statistic, decimals=3)}")
    print(f"p-value : {np.round(pr.pvalue, decimals=5)}")


rho = pr.statistic


Days of lag :  4
Pearson coefficient : -0.026
p-value : 0.07217

Days of lag :  3
Pearson coefficient : 0.015
p-value : 0.28638

Days of lag :  2
Pearson coefficient : -0.018
p-value : 0.21287

Days of lag :  1
Pearson coefficient : -0.07
p-value : 0.0

Days of lag :  0
Pearson coefficient : -0.296
p-value : 0.0


Lagging USDX does not improve the correlation between the time series.

The price adjustment of WTI is likely to happen intraday.

*Lagging WTI (not shown) does not improve the correlation either, which was expected.*

### Vector Autoregression model on the increments

In [ ]:
(osm_w, osr_w), (a_w, b_w, _) = probplot(wti_incr_unit, dist="norm", fit=True)
(osm_u, osr_u), (a_u, b_u, _) = probplot(usdx_incr_unit, dist="norm", fit=True)

In [ ]:
adf_wti, p_wti, lag_wti, *_ = adfuller(wti_incr_unit)
adf_usdx, p_usdx, lag_usdx, *_ = adfuller(usdx_incr_unit)
kpss_wti, pk_wti, lag_wti, *_ = kpss(wti_incr_unit)
kpss_usdx, pk_usdx, lag_usdx, *_ = kpss(usdx_incr_unit)

confidence_level = 0.01

print(f"""
WTI :
    Reject ADF : {p_wti < confidence_level}
    Reject KPSS : {pk_wti < confidence_level}

USDX :
    Reject ADF : {p_usdx < confidence_level}
    Reject KPSS : {pk_usdx < confidence_level}
""")


WTI :
    Reject ADF : True
    Reject KPSS : False

USDX :
    Reject ADF : True
    Reject KPSS : False



/tmp/ipykernel_6530/1819464982.py:3: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_wti, pk_wti, lag_wti, *_ = kpss(wti_incr_unit)
/tmp/ipykernel_6530/1819464982.py:4: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_usdx, pk_usdx, lag_usdx, *_ = kpss(usdx_incr_unit)


Both tests agree that the time series of increments are stationary.

From there it makes sense to fit an autoregressive model.

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(10, 6), sharey=True)

_ = plot_acf(wti_incr_unit, ax=axs[0][0], title="WTI autocorrelation", auto_ylims=True)
_ = plot_acf(
    usdx_incr_unit, ax=axs[0][1], title="USDX autocorrelation", auto_ylims=True
)
_ = plot_pacf(wti_incr_unit, ax=axs[1][0], title="WTI partial autocorrelation")
_ = plot_pacf(usdx_incr_unit, ax=axs[1][1], title="USDX partial autocorrelation")

fig.tight_layout()

In [ ]:
model = VAR(df_p)
var_res = model.fit()
var_res.summary()
print("Number of lags modelled :", var_res.k_ar)

Number of lags modelled : 1


/home/leonc/code/fixed-income-analytics/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/leonc/code/fixed-income-analytics/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [ ]:
r2_ar = 1.0 - var_res.resid.var() / df_p.var()
r2_ar.name = "R2 scores"
r2_ar

WTI spot    0.005867
USDX        0.001695
Name: R2 scores, dtype: float64

- WTI's increments variance is explained 5 times as much as USDX from the VAR model
- This corroborates the assumption that USDX drives WTI prices

In [ ]:
fitted_wti = (
    np.exp(var_res.fittedvalues["WTI spot"].cumsum()) * df_p["WTI spot"].std()
    + df.select(pl.col("WTI spot").first()).collect().item()
)  # mean.select(pl.col("WTI spot")).item()

fitted_wti = (
    fitted_wti + rho * df_p["USDX"] / df_p["USDX"].std() * df_p["WTI spot"].std()
)

In [ ]:
df_pd = df.collect().to_pandas().set_index("date")

usd_proj = (
    rho
    * (df_pd["USDX"] - df_pd["USDX"].mean())
    / df_pd["USDX"].std()
    * df_pd["WTI spot"].std()
)

In [ ]:
wti = (
    df.select([pl.col("WTI spot"), pl.col("date")])
    .collect()
    .to_pandas()
    .set_index("date")
)
_, ax = plt.subplots(figsize=(10, 4))
wti.plot(ax=ax)
_ = (usd_proj + fitted_wti).plot(ax=ax, grid=True)
_ = ax.set_title("WTI spot VAR + linear projection prediction")

In [ ]:
r2_var = 1 - (wti["WTI spot"] - fitted_wti).var() / wti["WTI spot"].var()
r2_var_lin_proj = (
    1 - (wti["WTI spot"] - fitted_wti - usd_proj).var() / wti["WTI spot"].var()
)

print(f"R2 VAR model : {np.round(r2_var, decimals=3)}")
print(f"R2 VAR model plus linear projection : {np.round(r2_var_lin_proj, decimals=3)}")

R2 VAR model : 0.273
R2 VAR model plus linear projection : 0.334


In [ ]:
wti_residuals = wti["WTI spot"] - fitted_wti - usd_proj

_ = wti_residuals.plot(
    figsize=(10, 4), grid=True, title="WTI residuals ie idiosyncratic price"
)

## Out-of-sample test

In [ ]:


adf_wti, p_wti, lag_wti, *_ = adfuller(wti_unit)
adf_usdx, p_usdx, lag_usdx, *_ = adfuller(usdx_unit)
kpss_wti, pk_wti, lag_wti, *_ = kpss(wti_unit)
kpss_usdx, pk_usdx, lag_usdx, *_ = kpss(usdx_unit)

confidence_level = 0.01

print(f"""
WTI :
    Reject ADF : {p_wti < confidence_level}
    Reject KPSS : {pk_wti < confidence_level}

USDX :
    Reject ADF : {p_usdx < confidence_level}
    Reject KPSS : {pk_usdx < confidence_level}
""")


WTI :
    Reject ADF : False
    Reject KPSS : False

USDX :
    Reject ADF : False
    Reject KPSS : False



/tmp/ipykernel_6530/2325128371.py:3: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_wti, pk_wti, lag_wti, *_ = kpss(wti_unit)
/tmp/ipykernel_6530/2325128371.py:4: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_usdx, pk_usdx, lag_usdx, *_ = kpss(usdx_unit)


In [ ]:
fig, ax = plt.subplots()
s = ax.scatter(usdx_unit, wti_unit, s=1., alpha=.8, c=range(len(usdx_unit)))
_ = fig.colorbar(s, ax=ax)
ax.set_xlabel("USDX")
ax.set_ylabel("WTI")
_ = plt.tight_layout()

***
## Sources

- https://open.spotify.com/episode/0LroaxByEYdHk5Bbzg7CMM

Data :
- https://finance.yahoo.com/quote/DX-Y.NYB/history/
- https://www.eia.gov/dnav/pet/hist/RWTCD.htm
